In [17]:
import numpy
# fine tune mt5 on dataset
from transformers import MT5ForConditionalGeneration, MT5Tokenizer, BertTokenizer, BertModel
from transformers import Trainer, TrainingArguments
from transformers import BertGenerationEncoder,BertGenerationDecoder,EncoderDecoderModel
from datasets import load_dataset
from transformers import DataCollatorForSeq2Seq
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from simpletransformers.t5 import T5Model, T5Args
from transformers import pipeline
#import train split
import pandas as pd
from sklearn.model_selection import train_test_split
import sklearn.preprocessing
import torch
import torch.nn as nn
from google.transliteration import transliterate_word
import klib
import os

In [2]:
#load dataset
#English-Hindi code-mixed parallel corpus.csv
df = pd.read_csv('../PHNC/English-Hindi code-mixed parallel corpus.csv')
df = df.dropna()
df = df.reset_index(drop=True)
# add column for prefix
df['prefix'] = 'translate English to Hinglish: '
df.head()


,Sentence,English_Translation,prefix
0,@someUSER congratulations on you celebrating b...,@some users congratulate you for celebrating B...,translate English to Hinglish:
1,@LoKarDi_RT uske liye toh bahot kuch karna pad...,"@Lokardi_ rat we should a lot more for that, b...",translate English to Hinglish:
2,@slimswamy yehi to hum semjhane ki koshish kar...,"@Slimswami ehi, this is what i'm expecting you...",translate English to Hinglish:
3,@DramebaazKudi cake kaha hai ??,@Where is Dramebajakudi where is the cake?,translate English to Hinglish:
4,@someUSER i'm in hawaii at the moment . home ...,@some user Don't want to come home next friday...,translate English to Hinglish:


In [3]:
#data cleaning 

df=klib.data_cleaning(df)

Shape of cleaned data: (13737, 2) - Remaining NAs: 0


Dropped rows: 1
     of which 1 duplicates. (Rows (first 150 shown): [1091])

Dropped columns: 1
     of which 1 single valued.     Columns: ['prefix']
Dropped missing values: 0
Reduced memory by at least: 0.1 MB (-32.26%)



In [4]:
#split train, val, test
# convert df  so that it can be used by transformers


train, test = train_test_split(df, test_size=0.2, random_state=42)
train, val = train_test_split(train, test_size=0.2, random_state=42)
train = train.reset_index(drop=True)
val = val.reset_index(drop=True)
test = test.reset_index(drop=True)

#print lens
print(len(train))
print(len(val))
print(len(test))

#save train, val, test
train.to_csv('../train.csv', index=False)
val.to_csv('../val.csv', index=False)
test.to_csv('..//test.csv', index=False)


8791
2198
2748


In [5]:
df.columns

Index(['sentence', 'english_translation'], dtype='object')

In [6]:
#tokenize
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
tokenizer.add_special_tokens({'additional_special_tokens': ['<sep>']})
tokenizer.add_special_tokens({'additional_special_tokens': ['<pad>']})
tokenizer.add_special_tokens({'additional_special_tokens': ['<s>']})
tokenizer.add_special_tokens({'additional_special_tokens': ['</s>']})
tokenizer.add_special_tokens({'additional_special_tokens': ['<unk>']})



1

In [7]:
df['sentence']

0        @someUSER congratulations on you celebrating b...
1        @LoKarDi_RT uske liye toh bahot kuch karna pad...
2        @slimswamy yehi to hum semjhane ki koshish kar...
3                          @DramebaazKudi cake kaha hai ??
4        @someUSER i'm in hawaii at the moment .  home ...
                               ...                        
13732    Dr Kumar Vishwas: "Koi deewana kehta hai.. koi...
13733    Me: Aaj kuch toofani karte hai.

Mom: Pani ki ...
13734    Pyar mangi to Jaan dengi,milk mango to kher de...
13735     @imcomplicated__ kaale kaale baal gaal gore gore
13736                            Ye sab aunty'on ke saath?
Name: sentence, Length: 13737, dtype: string

In [8]:
# Print the original sentence.
print(' Original: ', df['sentence'][0])

# Print the tweet split into tokens.
print('Tokenized: ', tokenizer.tokenize(df['sentence'][0]))

# Print the tweet mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(df['sentence'][0])))

 Original:  @someUSER congratulations on you celebrating british kid singers sophia grace's and rosie's 1st anniversary of a visit of your show .  how
Tokenized:  ['@', 'some', '##US', '##ER', 'con', '##grat', '##ulations', 'on', 'you', 'celebra', '##ting', 'brit', '##ish', 'ki', '##d', 'singers', 'so', '##phia', 'gra', '##ce', "'", 's', 'and', 'ros', '##ie', "'", 's', '1st', 'anniversary', 'of', 'a', 'visit', 'of', 'your', 'show', '.', 'how']
Token IDs:  [137, 11152, 32612, 24093, 10173, 84558, 74874, 10135, 13028, 42892, 12141, 52676, 15529, 10879, 10162, 81156, 10380, 89224, 63706, 10419, 112, 187, 10111, 39312, 10400, 112, 187, 13510, 37157, 10108, 169, 27541, 10108, 20442, 11897, 119, 14796]


In [9]:
maxlen = 512
def tokenize_df(df):
    target = tokenizer(df['sentence'], padding='max_length', truncation=True, return_tensors="pt", max_length=maxlen)
    input = tokenizer(df['english_translation'], padding='max_length', truncation=True, return_tensors="pt", max_length=maxlen)
    input_ids = input['input_ids']
    attention_mask = input['attention_mask']
    target_ids = target['input_ids']
    target_attention_mask = target['attention_mask']
    decoder_input_ids = target_ids.clone()
    #convert to tensors
    input_ids = torch.tensor(input_ids).squeeze()
    attention_mask = torch.tensor(attention_mask).squeeze()
    target_ids = torch.tensor(target_ids).squeeze()
    target_attention_mask = torch.tensor(target_attention_mask).squeeze()
   # decoder_input_ids = torch.tensor(decoder_input_ids)
    
    return {
        'input_ids': input_ids,
        'attention_mask': attention_mask,
        'labels': target_ids,
        #'decoder_input_ids': decoder_input_ids,
        #'decoder_attention_mask': target_attention_mask
    }


In [10]:
#tokenize train, val, test
train = load_dataset('csv', data_files='train.csv')
val = load_dataset('csv', data_files='val.csv')
test = load_dataset('csv', data_files='test.csv')
train = train.map(tokenize_df, batched=True, batch_size=128,remove_columns=['sentence','english_translation'])
val = val.map(tokenize_df, batched=True, batch_size=128,remove_columns=['sentence','english_translation'])
test = test.map(tokenize_df, batched=True, batch_size=128,remove_columns=['sentence','english_translation'])


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 966.21it/s]
Generating train split: 0 examples [00:00, ? examples/s]/home/aparna/miniconda3/lib/python3.10/site-packages/datasets/download/streaming_download_manager.py:776: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Dataset csv downloaded and prepared to /home/aparna/.cache/huggingface/datasets/csv/default-02482a7dc6cd63f0/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 400.53it/s]


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 655.05it/s]
Generating train split: 0 examples [00:00, ? examples/s]/home/aparna/miniconda3/lib/python3.10/site-packages/datasets/download/streaming_download_manager.py:776: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Dataset csv downloaded and prepared to /home/aparna/.cache/huggingface/datasets/csv/default-a6c55461f7455f05/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 394.68it/s]


Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 753.15it/s]
Generating train split: 0 examples [00:00, ? examples/s]/home/aparna/miniconda3/lib/python3.10/site-packages/datasets/download/streaming_download_manager.py:776: FutureWarning: the 'mangle_dupe_cols' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'mangle_dupe_cols'
  return pd.read_csv(xopen(filepath_or_buffer, "rb", use_auth_token=use_auth_token), **kwargs)


Dataset csv downloaded and prepared to /home/aparna/.cache/huggingface/datasets/csv/default-18d6eb5fdf4808e6/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


Map:   0%|          | 0/8791 [00:00<?, ? examples/s]/tmp/ipykernel_141077/3378448442.py:11: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(input_ids).squeeze()
/tmp/ipykernel_141077/3378448442.py:12: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_mask = torch.tensor(attention_mask).squeeze()
/tmp/ipykernel_141077/3378448442.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  target_ids = torch.tensor(target_ids).squeeze()
/tmp/ipykernel_141077/3378448442.py:14: UserWarning: To copy construct from a tensor

In [11]:
train
#get sample 
sample = train['train'][0]
sample
#print shapes
print(len(sample['input_ids']))
print(len(sample['attention_mask']))
#print(len(sample['decoder_input_ids']))
#print(len(sample['decoder_attention_mask']))


512
512


In [12]:
val

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2198
    })
})

In [13]:
# from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
# batch_size = 8
# train_dataloader = DataLoader(

#             train,  # The training samples.

#             sampler = RandomSampler(train), # Select batches randomly

#             batch_size = batch_size # Trains with this batch size.

#         )

# validation_dataloader = DataLoader(

#             val, # The validation samples.

#             sampler = SequentialSampler(val), # Pull out batches sequentially.

#             batch_size = batch_size # Evaluate with this batch size.

#         )

# test_dataloader = DataLoader(


#             test, # The validation samples. 

#             sampler = SequentialSampler(test), # Pull out batches sequentially.

#             batch_size = batch_size # Evaluate with this batch size.

#         )

# #test train data loader
# for batch in train_dataloader:

#     print(batch)

#     break


In [14]:
import evaluate
import numpy as np
from nltk.tokenize import RegexpTokenizer

rouge_metric = evaluate.load("rouge")

def tokenize_sentence(arg):
    encoded_arg =tokenizer(arg)
    return tokenizer.convert_ids_to_tokens(encoded_arg.input_ids)

def metrics_func(eval_arg):
    preds, labels = eval_arg
    # Replace -100
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    # Convert id tokens to text
    text_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    text_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Insert a line break (\n) in each sentence for ROUGE scoring
    # (Note : Please change this code, when you perform on other languages except for Japanese)
    text_preds = [(p if p.endswith(("!", "！", "?", "？", "。")) else p + "。") for p in text_preds]
    text_labels = [(l if l.endswith(("!", "！", "?", "？", "。")) else l + "。") for l in text_labels]
    sent_tokenizer_jp = RegexpTokenizer(u'[^!！?？。]*[!！?？。]')
    text_preds = ["\n".join(np.char.strip(sent_tokenizer_jp.tokenize(p))) for p in text_preds]
    text_labels = ["\n".join(np.char.strip(sent_tokenizer_jp.tokenize(l))) for l in text_labels]
    # compute ROUGE score with custom tokenization
    return rouge_metric.compute(
        predictions=text_preds,
        references=text_labels,
        tokenizer=tokenize_sentence
    )

In [19]:
# finetuen mt5
os.environ["WANDB_DISABLED"] = "true"

#model = BertGenerationEncoder.from_pretrained("bert-base-multilingual-cased")


encoder = BertGenerationEncoder.from_pretrained("bert-base-multilingual-cased", bos_token_id=101, eos_token_id=102)
# add cross attention layers and use BERT's cls token as BOS token and sep token as EOS token
decoder = BertGenerationDecoder.from_pretrained(
    "bert-large-uncased", add_cross_attention=True, is_decoder=True, bos_token_id=101, eos_token_id=102
)
model = EncoderDecoderModel(encoder=encoder, decoder=decoder)

model.encoder.resize_token_embeddings(len(tokenizer))
model.decoder.resize_token_embeddings(len(tokenizer))


Embedding(119552, 1024)

In [ ]:


#training args


training_args = Seq2SeqTrainingArguments(
  output_dir = "mbert-codemixed",
  log_level = "error",
  num_train_epochs = 10,
  learning_rate = 5e-4,
  lr_scheduler_type = "linear",
  warmup_steps = 90,
  optim = "adafactor",
  weight_decay = 0.01,
  per_device_train_batch_size =2,
  per_device_eval_batch_size = 1,
  gradient_accumulation_steps = 16,
  evaluation_strategy = "steps",
  eval_steps = 100,
  predict_with_generate=True,
  generation_max_length = 128,
  save_steps = 500,
  logging_steps = 10,
  push_to_hub = False
)


#trainer
trainer = Seq2SeqTrainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train["train"],         # training dataset
    eval_dataset=val["train"],             # evaluation dataset
    tokenizer=tokenizer,               # tokenizer
    data_collator=DataCollatorForSeq2Seq(tokenizer, model=model), # data collator
    
)

#train
trainer.train()

#save model
trainer.save_model("./mbert")

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
  0%|          | 0/2740 [00:00<?, ?it/s]

ValueError: The model did not return a loss from the inputs, only the following keys: last_hidden_state. For reference, the inputs it received are input_ids,attention_mask.

In [ ]:
from torch.utils.data import DataLoader
model = MT5ForConditionalGeneration.from_pretrained("./mt5v1")
#tokenizer = MT5Tokenizer.from_pretrained("./mt5")


sample_dataloader = DataLoader(
  test["train"].with_format("torch"),
  collate_fn=DataCollatorForSeq2Seq(tokenizer, model=model),
  batch_size=5)
for batch in sample_dataloader:
  with torch.no_grad():
    preds = model.generate(
      batch["input_ids"],
      num_beams=15,
      num_return_sequences=1,
      no_repeat_ngram_size=1,
      remove_invalid_values=True,
      max_length=128,
    )
  labels = batch["labels"]
  break
print(preds, labels)
metrics_func([preds, labels])

tensor([[     0,   1250,   7800, 198280,    259,  65334,    260,   1061,  39084,
          19349,    318,  51571,  62342,  16263, 144044,    603,   2148,    513,
           2941,    334,   1312,  88806,   2050,    432,    262,    268,    387,
           1759,    290, 165794,      1,      0,      0,      0,      0,      0,
              0,      0,      0],
        [     0,  27696,  62342,    259,    261,    342,    776, 113865,    714,
           2829,    387, 102339,   9065,  42716,    260,   1816,    321,  12961,
            623,   3663,    479,   1776,   1250,   6253, 182594,    262,    290,
           1795,   1061, 146525,  56696,    313,  11395,    330,  35714,    609,
           1350,    311,      1],
        [     0,   1250,  38393,    265,    299,    609,    339,   4592,   6504,
            259,   1542,    787,   3007,    288,   6313,    260,   1061,    559,
            604,    263, 152418,   7925,   7954,      1,      0,      0,      0,
              0,      0,      0,      0, 

{'rouge1': 0.47184469277492536,
 'rouge2': 0.32362358952522885,
 'rougeL': 0.44858887882143694,
 'rougeLsum': 0.44858887882143694}

In [ ]:
from torch.utils.data import DataLoader

# Predict with test data (first 5 rows)
sample_dataloader = DataLoader(
  test["train"].with_format("torch"),
  collate_fn=DataCollatorForSeq2Seq(tokenizer, model=model),
  batch_size=5)
for batch in sample_dataloader:
  with torch.no_grad():
    preds = model.generate(
      batch["input_ids"],
      num_beams=15,
      num_return_sequences=1,
      no_repeat_ngram_size=1,
      remove_invalid_values=True,
      max_length=128,
    )
  labels = batch["labels"]
  inputs = batch["input_ids"]
  break

# Replace -100 (see above)
inputs = np.where(inputs != -100, inputs, tokenizer.pad_token_id)
labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

# Convert id tokens to text
text_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
text_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
text_inputs = tokenizer.batch_decode(inputs, skip_special_tokens=True)

# Show result
print("***** Input's Text *****")
print(text_inputs[2])
print("***** codemix (True Value) *****")
print(text_labels[2])
print("***** codemix (Generated Text) *****")
print(text_preds[2])

***** Input's Text *****
@rynkee it is thi thought which we want to change. @PunsTurnMeOn
***** codemix (True Value) *****
@rynkee yehi soch to badalni hai @PunsTurnMeOn
***** codemix (Generated Text) *****
@rynkees it is thi thought which we want to change.@PunsTurnMeOn


In [ ]:
for i in range(5):
    print("***** Input's Text *****")
    print(text_inputs[i])
    print("***** codemix (True Value) *****")
    print(text_labels[i])
    print("***** codemix (Generated Text) *****")
    print(text_preds[i])

***** Input's Text *****
@hurdangi haan.. @sagarikaghose sister will eat green mango today @the_hindu
***** codemix (True Value) *****
@hurdangi haan.. @sagarikaghose Didi aaj hare rang ke aam khaengi @the_hindu
***** codemix (Generated Text) *****
@hurdangi haan.@sagarikaghose bhai green mango peene ke saath kharab kar jaao #the_hindu
***** Input's Text *****
wait brother, do not cry this much, its #GST not a bomb. have some shame. @digvijaya_28 @INCIndia " country brought it out "now you sit and cry
***** codemix (True Value) *****
Are bas kar bhai itna nahi rone "ka #GST hai bomb nahi. Kuch to sharm karo. @digvijaya_28 @INCIndia " desh nikal liya "aage u sit and cry
***** codemix (Generated Text) *****
wait bhai, do not cry this much #GST nahi bomb. Haan kuch ho chuka hai @digvijaya_28@INCIndia " country brought it out"
***** Input's Text *****
@rynkee it is thi thought which we want to change. @PunsTurnMeOn
***** codemix (True Value) *****
@rynkee yehi soch to badalni hai @PunsTurn